In [22]:
import pandas as pd
import time
from graphviz import Digraph


# Binary Search Tree Node
class BSTNode:
    def __init__(self, key, data):
        self.key = key  # Customer ID
        self.data = data  # Dictionary with customer details
        self.left = None
        self.right = None


# Binary Search Tree Class
class BST:
    def __init__(self):
        self.root = None

    def get_height(self, node):
        if not node:
            return 0
        return 1 + max(self.get_height(node.left), self.get_height(node.right))

    def insert(self, node, key, data):
        if not node:
            return BSTNode(key, data)
        if key < node.key:
            node.left = self.insert(node.left, key, data)
        elif key > node.key:
            node.right = self.insert(node.right, key, data)
        else:
            raise ValueError(f"Duplicate key '{key}' is not allowed!") #hundiling duplicates
        return node

    def search(self, node, key):
        if not node or node.key == key:
            return node
        if key < node.key:
            return self.search(node.left, key)
        return self.search(node.right, key)

    def delete(self, node, key):
        if not node:
            return node

        if key < node.key:
            node.left = self.delete(node.left, key)
        elif key > node.key:
            node.right = self.delete(node.right, key)
        else:
            if not node.left:
                return node.right
            elif not node.right:
                return node.left

            temp = self.get_min_value_node(node.right)
            node.key = temp.key
            node.data = temp.data
            node.right = self.delete(node.right, temp.key)

        return node

    def get_min_value_node(self, node):
        current = node
        while current and current.left:
            current = current.left
        return current

    def insert_customer(self, key, data):
        self.root = self.insert(self.root, key, data)

    def search_customer(self, key):
        node = self.search(self.root, key)
        return node.data if node else None

    def delete_customer(self, key):
        self.root = self.delete(self.root, key)

    def is_balanced(self):
        def check_balance(node):
            if not node:
                return True, 0  # Balanced and height 0
            left_balanced, left_height = check_balance(node.left)
            right_balanced, right_height = check_balance(node.right)
            balance_factor = abs(left_height - right_height)
            is_balanced = left_balanced and right_balanced and balance_factor <= 1
            return is_balanced, 1 + max(left_height, right_height)

        balanced, _ = check_balance(self.root)
        return balanced

    def visualize(self):
        """
        Visualize the BST using Graphviz.
        """
        dot = Digraph()
        self._add_nodes(dot, self.root)
        return dot

    def _add_nodes(self, dot, node):
        if node:
            dot.node(str(node.key), f"{node.key}\n{node.data['First Name']} {node.data['Last Name']}")
            if node.left:
                dot.edge(str(node.key), str(node.left.key))
                self._add_nodes(dot, node.left)
            if node.right:
                dot.edge(str(node.key), str(node.right.key))
                self._add_nodes(dot, node.right)


# Load dataset into BST
def load_data_to_bst(tree, filepath):
    customer_data = pd.read_csv(filepath)
    for _, row in customer_data.iterrows():
        customer_id = row["Customer Id"]
        customer_info = {
            "First Name": row["First Name"],
            "Last Name": row["Last Name"],
            "Company": row["Company"],
            "City": row["City"],
            "Country": row["Country"],
            "Phone 1": row["Phone 1"],
            "Phone 2": row["Phone 2"],
            "Email": row["Email"],
            "Subscription Date": row["Subscription Date"],
            "Website": row["Website"],
        }
        tree.insert_customer(customer_id, customer_info)


# Process commands from a file
def process_commands(filename, tree):
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',', 2)
            command = parts[0]
            key = parts[1]
            if command == "INSERT":
                # Convert string to dictionary using eval
                data = eval(parts[2])
                tree.insert_customer(key, data)
            elif command == "SEARCH":
                result = tree.search_customer(key)
                if result:
                    print(f"Found: {result}")
                else:
                    print("Not found.")
            elif command == "DELETE":
                tree.delete_customer(key)
                print(f"Deleted customer with ID {key}.")


def measure_time(operation, *args):
    start_time = time.perf_counter() 
    result = operation(*args)
    end_time = time.perf_counter()
    elapsed_time_ms = (end_time - start_time) * 1000  # Convert seconds to milliseconds
    return elapsed_time_ms, result

# Main Program
if __name__ == "__main__":
    # Initialize BST
    customer_tree = BST()

    # Load data into BST
    data_file = 'customers-100000.csv'
    print("Loading data into BST...")
    load_data_to_bst(customer_tree, data_file)
    print("Data loaded successfully.")

    # Example: Search operation
    print("\nExample: Search for a customer.")
    search_key = "10dAcafEBbA5Fczzz"
    search_time, search_result = measure_time(customer_tree.search_customer, search_key)
    if search_result:
        print(f"Customer Found: {search_result} (Time taken: {search_time:.4f} milliseconds)")
    else:
        print(f"Customer with ID {search_key} not found. (Time taken: {search_time:.4f} milliseconds)")

    # Example: Insert operation
    print("\nExample: Insert a new customer.")
    new_customer_id = "10dAcafEBbA5FcZ"
    new_customer_data = {
        "First Name": "mohammad",
        "Last Name": "al rifai",
        "Company": "Scale ai",
        "City": "sharjah",
        "Country": "UAE",
        "Phone 1": "054-329-7312",
        "Phone 2": "098-765-4321",
        "Email": "Mohammad_alrifai@example.com",
        "Subscription Date": "2024-11-25",
        "Website": "https:/alrifai.com",
    }
    insert_time, _ = measure_time(customer_tree.insert_customer, new_customer_id, new_customer_data)
    print(f"Inserted new customer with ID {new_customer_id}. (Time taken: {insert_time:.4f} milliseconds)")

    # Example: Delete operation
    print("\nExample: Delete a customer.")
    delete_key = "10dAcafEBbA5FcA"
    delete_time, _ = measure_time(customer_tree.delete_customer, delete_key)
    print(f"Deleted customer with ID {delete_key}. (Time taken: {delete_time:.4f} milliseconds)")



    # Check tree balance
    if customer_tree.is_balanced():
        print("\nThe BST is balanced.")
    else:
        print("\nThe BST is NOT balanced.")

    # Visualize the tree
    #tree_dot = customer_tree.visualize()
    #tree_dot.render('bst_tree', format='pdf', cleanup=True)  # Save as PDF
    #print("Tree visualization saved as 'bst_tree.pdf'.")


Loading data into BST...
Data loaded successfully.

Example: Search for a customer.
Customer with ID 10dAcafEBbA5Fczzz not found. (Time taken: 0.0240 milliseconds)

Example: Insert a new customer.
Inserted new customer with ID 10dAcafEBbA5FcZ. (Time taken: 0.0061 milliseconds)

Example: Delete a customer.
Deleted customer with ID 10dAcafEBbA5FcA. (Time taken: 0.0069 milliseconds)

The BST is NOT balanced.
